In [6]:
import math
import itertools
import numpy as np
import matplotlib.pyplot as plt

from functools import reduce

from utils import *

# %matplotlib inline

In [5]:
print(num_admissible_poolings(2, 5, 6))

20
